In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [13]:
# CUDA 사용 대비, 넣어두길 추천



USE_CUDA=torch.cuda.is_available()     # -> CUDA용 PYTORCH와 CUDA설치시 TRUE반환


DEVICE=torch.device('cuda' if USE_CUDA else 'cpu')

# 데이터셋 불러오기

In [14]:
EPOCHS=30
BATCH_SIZE=64

transform=transforms.Compose([transforms.ToTensor()])

trainset=datasets.FashionMNIST(
    root='./.data/',
    train=True,
    download=True,
    transform=transform
)

testset=datasets.FashionMNIST(
    root='./.data/',
    train=False,
    download=True,
    transform=transform
)

train_loader=torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

train_loader=torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader=torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# 신경망 설계

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,256)  # nn.Linear = Fully connected Layer(Dense)
        self.fc2=nn.Linear(256,128)
        self.fc3=nn.Linear(128,10)   # 최종적으로 10개를 출력(패션아이템 카테고리 갯수) 
        
        
    def forward(self, x):
        x=x.view(-1,784)             # 1차원 행렬로 reshape 
        x=F.relu(self.fc1(x))        # Relu함수
        x=F.relu(self.fc2(x))
        x=self.fc3(x)                # 출력 10개의 값
        return x
        

# 모델 선언

In [27]:
model=Net().to(DEVICE)  # 신경망 모델 선언
optimizer=optim.SGD(model.parameters(), lr=0.01) # SGD로 optimizer설정, learning rate=0.01

'''
 Loss Function을 계산할 때 전제 Train-Set을 사용하는 것을 Batch Gradient Descent라고 한다. 그러나 이렇게 계산하면 
 한번 step을 내딛을 때, 전체 데이터에 대해 Loss Function을 계산해야 하므로 너무 많은 계산양을 필요로 한다. 이를 방지하기 
 위해 보통은 Stochastic Gradient Desenct(SGD)라는 방법을 사용한다. 
 이 방법에서는 Loss Function을 계산할 때, 전체 데이터(Batch) 대신 일부 데이터의 모음(Mini-Batch)를 사용하여 Loss Function을 계산
'''

'\n Loss Function을 계산할 때 전제 Train-Set을 사용하는 것을 Batch Gradient Descent라고 한다. 그러나 이렇게 계산하면 \n 한번 step을 내딛을 때, 전체 데이터에 대해 Loss Function을 계산해야 하므로 너무 많은 계산양을 필요로 한다. 이를 방지하기 \n 위해 보통은 Stochastic Gradient Desenct(SGD)라는 방법을 사용한다. \n 이 방법에서는 Loss Function을 계산할 때, 전체 데이터(Batch) 대신 일부 데이터의 모음(Mini-Batch)를 사용하여 Loss Function을 계산\n'

# 학습 모델 설계

In [28]:
def train(model, train_loader, optimizer):
    model.train()           # 모델을 학습모드로 선언 -> 학습/평가모드에 따라 동작이 다른 함수(Like. Dropout)의 오동작을 예방
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target=data.to(DEVICE), target.to(DEVICE),    # 학습데이터를 기기메모리로 보냄
        optimizer.zero_grad()                               # 기울기 초기화, 반복때마다 기울기를 새로 계산하기위해 호출
        output = model(data)                                # to()함수로 모델가중치에 사용한 같은 장치(CPU OR GPU) 이용
        loss=F.cross_entropy(output, target)                # 예측값(Output)과 레이블값(Target값)오차 구하기 (교차 엔트로피-Cross Entropy 이용)
        loss.backward() 
        optimizer.step()                                    # 계산한 기울기를 모델에 맞춰 가중치 수정
        

# 딥러닝 모델 성능 테스트

In [29]:
def evaluate(model, test_loader):
    model.eval()     # 모델을 평가모드로 전환
    test_loss=0      # 
    correct=0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output=model(data)
            
            test_loss+=F.cross_entropy(output, target, reduction='sum').item()   # 모든 미니배치의 Loss합 = test_loss값
            
            # 가장 큰값을 가진 클래스가 모델의 예측값
            # 예측과 정답을 비교하여 일치할 경우, correct에 1을 더함
            pred=output.max(1,keepdim=True)[1]                        
            correct += pred.eq(target.view_as(pred)).sum().item()     # 'Sun90'으로 배치에서 모델이 정답을 맞춘 갯수구함
            
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy
            

# 모델 확인

In [30]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss : 0.8409, Accuracy : 68.43%
[2] Test Loss : 0.6731, Accuracy : 75.74%
[3] Test Loss : 0.5892, Accuracy : 78.96%
[4] Test Loss : 0.5687, Accuracy : 79.38%
[5] Test Loss : 0.5209, Accuracy : 81.38%
[6] Test Loss : 0.4999, Accuracy : 82.38%
[7] Test Loss : 0.4906, Accuracy : 82.38%
[8] Test Loss : 0.4802, Accuracy : 83.09%
[9] Test Loss : 0.4764, Accuracy : 83.27%
[10] Test Loss : 0.4608, Accuracy : 83.86%
[11] Test Loss : 0.4643, Accuracy : 83.69%
[12] Test Loss : 0.4632, Accuracy : 83.54%
[13] Test Loss : 0.4484, Accuracy : 84.00%
[14] Test Loss : 0.4337, Accuracy : 84.55%
[15] Test Loss : 0.4237, Accuracy : 84.87%
[16] Test Loss : 0.4189, Accuracy : 85.00%
[17] Test Loss : 0.4302, Accuracy : 84.72%
[18] Test Loss : 0.4243, Accuracy : 84.77%
[19] Test Loss : 0.4115, Accuracy : 85.43%
[20] Test Loss : 0.3972, Accuracy : 85.88%
[21] Test Loss : 0.4265, Accuracy : 84.33%
[22] Test Loss : 0.4058, Accuracy : 85.59%
[23] Test Loss : 0.3941, Accuracy : 86.21%
[24] Test Loss : 0.4